In [24]:
import numpy as np
import time
from scipy.optimize import minimize

In [ ]:
Вводы-data-as-joinbyfield-2024-07-23 12_26_58.csv

In [25]:
# Целевая функция
def task_func(x):
    return x[0] ** 2 + x[1] ** 2

# Градиент целевой функции
def gradient(x):
    return np.array([2 * x[0], 2 * x[1]])

# Ограничения
def cons1(x):
    return -(2 * x[0] + x[1] - 2)

def cons2(x):
    return -(-x[1] + 1)

# Проекция на область допустимых значений
def proj(x):
    if cons1(x) < 0:
        # Проекция на -2 * x[0] - x[1] + 2 >= 0
        a, b, c = -2, -1, 2
        x = x - ((a * x[0] + b * x[1] - c) / (a ** 2 + b ** 2)) * np.array([a, b])
    if cons2(x) < 0:
        # Проекция на x[1] >= 1
        x[1] = max(x[1], 1)
    return x

# Штрафная функция
def penalty_func(x, mu):
    penalty = 0
    if -(cons1(x)) > 0:
        penalty += mu * cons1(x) ** 2
    if -(cons2(x)) > 0:
        penalty += mu * cons2(x) ** 2
    return penalty

# Общая функция с штрафом
def total_func(x, mu):
    return task_func(x) + penalty_func(x, mu)

# Градиент общей функции с штрафом
def total_gradient(x, mu):
    grad = gradient(x)
    if -(cons1(x)) > 0:
        grad += 2 * mu * -(cons1(x)) * np.array([2, 1])
    if -(cons2(x)) > 0:
        grad += 2 * mu * -(cons2(x)) * np.array([0, -1])
    return grad

In [26]:
# Параметры Adam
beta1 = 0.9  # коэффициент экспоненциального затухания для момента
beta2 = 0.999  # коэффициент экспоненциального затухания для скорости
eps = 1e-8  # погрешность
max_iter = 1000  # максимальное количество итераций
mu1 = 1.0    # параметры штрафа
mu2 = 10.0   # параметры штрафа

# Начальная точка
x = np.array([2.0, 6.0])
x0 = x

# Инициализация моментов
m = np.zeros_like(x)
v = np.zeros_like(x)

---

## <center>Adam μ = 1 </center>

In [27]:
# Таймер работы Adam
start_timer_adam = time.perf_counter()

iter_count = 0

# Adam
for i in range(max_iter):
    grad = total_gradient(x, mu1)
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * grad**2
    m_hat = m / (1 - beta1**(i+1))
    v_hat = v / (1 - beta2**(i+1))
    new_x = x - 1 * m_hat / (np.sqrt(v_hat) + eps)

    if np.linalg.norm(new_x - x) < eps:
        iter_count = i
        break

    x = new_x

end_timer_adam = time.perf_counter()

In [28]:
# Scipy.minimize
cons = [{'type': 'ineq', 'fun': cons1},
        {'type': 'ineq', 'fun': cons2}]

start_timer_scipy = time.perf_counter()

result_scipy = minimize(task_func, x0, constraints=cons)

end_timer_scipy = time.perf_counter()

In [29]:
# Вывод результатов
print("Adam\n", "Корень:", x)
print(" Значение целевой функции:", task_func(x), "\n")

print("Scipy\n", "Корень:", result_scipy.x)
print(" Значение целевой функции:", result_scipy.fun, "\n")

print("Погрешность:", abs(result_scipy.fun - task_func(x)))
print("Время работы Adam:", round(end_timer_adam - start_timer_adam, 5), "секунд")
print("Время работы библиотеки scipy:", round(end_timer_scipy - start_timer_scipy, 5), "секунд")

print("Количество шагов выполнения Adam:", iter_count)

Adam
 Корень: [7.14650843e-08 5.00000079e-01]
 Значение целевой функции: 0.2500000788377943 

Scipy
 Корень: [-5.96046423e-09  1.00000000e+00]
 Значение целевой функции: 0.9999999999999976 

Погрешность: 0.7499999211622033
Время работы Adam: 0.00642 секунд
Время работы библиотеки scipy: 0.00187 секунд
Количество шагов выполнения Adam: 343


---

## <center>Adam μ = 10</center>

In [30]:
# Начальная точка
x = np.array([2.0, 6.0])
x0 = x

# Инициализация моментов
m = np.zeros_like(x)
v = np.zeros_like(x)

# Таймер работы Adam
start_timer_adam = time.perf_counter()

iter_count = 0

# Adam
for i in range(max_iter):
    grad = total_gradient(x, mu2)
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * grad**2
    m_hat = m / (1 - beta1**(i+1))
    v_hat = v / (1 - beta2**(i+1))
    new_x = x - 1 * m_hat / (np.sqrt(v_hat) + eps)

    if np.linalg.norm(new_x - x) < eps:
        iter_count = i
        break

    x = new_x

end_timer_adam = time.perf_counter()

# Вывод результатов
print("Adam\n", "Корень:", x)
print(" Значение целевой функции:", task_func(x), "\n")

print("Scipy\n", "Корень:", result_scipy.x)
print(" Значение целевой функции:", result_scipy.fun, "\n")

print("Погрешность:", abs(result_scipy.fun - task_func(x)))
print("Время работы Adam:", round(end_timer_adam - start_timer_adam, 5), "секунд")
print("Время работы библиотеки scipy:", round(end_timer_scipy - start_timer_scipy, 5), "секунд")

print("Количество шагов выполнения Adam:", iter_count)

Adam
 Корень: [2.49432638e-07 9.09090864e-01]
 Значение целевой функции: 0.8264461996925012 

Scipy
 Корень: [-5.96046423e-09  1.00000000e+00]
 Значение целевой функции: 0.9999999999999976 

Погрешность: 0.17355380030749634
Время работы Adam: 0.00484 секунд
Время работы библиотеки scipy: 0.00187 секунд
Количество шагов выполнения Adam: 352


-----

## <center>Adam без штрафной функции</center>

In [31]:
# Начальная точка
x = np.array([2.0, 6.0])
x0 = x

# Инициализация моментов
m = np.zeros_like(x)
v = np.zeros_like(x)

# Таймер работы Adam
start_timer_adam = time.perf_counter()

iter_count = 0

# Adam
for i in range(max_iter):
    grad = gradient(x)
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * grad**2
    m_hat = m / (1 - beta1**(i+1))
    v_hat = v / (1 - beta2**(i+1))
    new_x = proj(x - 1 * m_hat / (np.sqrt(v_hat) + eps))

    if np.linalg.norm(new_x - x) < eps:
        iter_count = i
        break

    x = new_x

end_timer_adam = time.perf_counter()

# Вывод результатов
print("Adam\n", "Корень:", x)
print(" Значение целевой функции:", task_func(x), "\n")

print("Scipy\n", "Корень:", result_scipy.x)
print(" Значение целевой функции:", result_scipy.fun, "\n")

print("Погрешность:", abs(result_scipy.fun - task_func(x)))
print("Время работы Adam:", round(end_timer_adam - start_timer_adam, 5), "секунд")
print("Время работы библиотеки scipy:", round(end_timer_scipy - start_timer_scipy, 5), "секунд")

print("Количество шагов выполнения Adam:", iter_count)

Adam
 Корень: [-1.3872228e-05  1.0000000e+00]
 Значение целевой функции: 1.0000000001924387 

Scipy
 Корень: [-5.96046423e-09  1.00000000e+00]
 Значение целевой функции: 0.9999999999999976 

Погрешность: 1.9244117410721628e-10
Время работы Adam: 0.00265 секунд
Время работы библиотеки scipy: 0.00187 секунд
Количество шагов выполнения Adam: 228
